Librerias

In [1]:
import numpy as np
import pandas as pd
import ast #libreria para la lectura literal de la linea

Lectura austrlian_user_reviews.json

In [2]:
list = []
for line in open('C:\\Users\\Usuario\\Desktop\\PI_MLOps\\dataset_original\\australian_user_reviews.json', encoding='utf-8').readlines():
    list.append(ast.literal_eval(line))
df = pd.DataFrame.from_dict(list)

Transform dataframe

In [4]:
#Create df_User_Reviews'
df_User_Reviews = df[['user_id','reviews']]

Transform column 'reviews'

In [5]:
#Expand 'reviews'
df_User_Reviews = df_User_Reviews.explode('reviews')

In [6]:
#Reset index 
df_User_Reviews.reset_index(drop=True, inplace=True)

In [7]:
#Delete nulls  
df_User_Reviews = df_User_Reviews.dropna()

In [8]:
#Delete keys 'last_edited' and 'funny'
df_User_Reviews['reviews'].apply(lambda x : x.pop('last_edited'))
df_User_Reviews['reviews'].apply(lambda x : x.pop('funny'))

0                                        
1                                        
2                                        
3                                        
4                                        
                       ...               
59328                                    
59329                                    
59330    1 person found this review funny
59331                                    
59332                                    
Name: reviews, Length: 59305, dtype: object

In [9]:
#Change type of data for key 'posted' for each review
def clean_Posted(dic):
    #Replace 'Posted' --> ''
    dic['posted'] = dic['posted'].replace('Posted ','')
    #Replace '.' --> ''
    dic['posted'] = dic['posted'].replace('.','')

df_User_Reviews['reviews'].apply(lambda row: clean_Posted(row))

0        None
1        None
2        None
3        None
4        None
         ... 
59328    None
59329    None
59330    None
59331    None
59332    None
Name: reviews, Length: 59305, dtype: object

In [10]:
#Convert to datetime

#Import class datetime
from datetime import datetime

def format(dic):
    try:
        #Try to convert
        dic['posted']=(datetime.strptime(dic['posted'],'%B %d, %Y')).strftime('%d-%m-%Y')
        return dic
    except ValueError:
        #catch error
        return None
    
df_User_Reviews['reviews']=df_User_Reviews['reviews'].apply(lambda row: format(row))

In [11]:
#Elimino NaNs 
df_User_Reviews.dropna(inplace=True)

In [12]:
#Convert to int 
def to_Int(dic):
    try:
        #Try to convert
        dic['item_id'] = int(dic['item_id']) 
        return dic
    except ValueError:
        #catch error
        return None
    
df_User_Reviews['reviews'] = (df_User_Reviews['reviews'].apply(lambda row: to_Int(row))).dropna()

In [13]:
#Modifico tipo de dato boolean --> int sobre la columna 'recommend' 
def to_Int_Boolean(dic):
    try:
        #Try to convert
        if (dic['recommend']):
            #recommend == True
            dic['recommend'] = 1
        else:
            #Recommen == False
            dic['recommend'] = 0
    except ValueError:
        #catch error
        return None

df_User_Reviews['reviews'].apply(lambda row: to_Int_Boolean(row))

0        None
1        None
2        None
3        None
4        None
         ... 
59280    None
59283    None
59293    None
59295    None
59304    None
Name: reviews, Length: 49186, dtype: object

In [16]:
#Elimino NaNs 
df_User_Reviews.dropna(inplace=True)

Feature Engineering

Consultas df_User_Reviews

In [15]:
#Tip de dato de la clave 'recommend'
type(df_User_Reviews['reviews'][0]['recommend'])

int

Lectura australian_users_items.json

In [2]:
list = []
for line in open('C:\\Users\\Usuario\\Desktop\\PI_MLOps\\dataset_original\\australian_users_items.json', encoding='utf-8').readlines():
    list.append(ast.literal_eval(line))
df = pd.DataFrame.from_dict(list)

Transform DataFrame

In [3]:
#Create df_Users_Items'
df_Users_Items = df[['user_id','items_count','items']]

In [4]:
#Expand 'items'
df_Users_Items=df_Users_Items.explode('items')

In [5]:
#Reset Index
df_Users_Items.reset_index(drop=True, inplace=True)

In [6]:
#Cantidad de filas con datos NULLS en la columna 'items'
df_Users_Items['items'].isna().sum()

16806

In [7]:
#Elimino Nulls  
df_Users_Items.dropna(inplace=True) 

In [10]:
#Elimino key 'playtime_2weeks'
df_Users_Items['items'].apply(lambda x : x.pop('playtime_2weeks'))

0          0
1          0
2          0
3          0
4          0
          ..
5170009    0
5170010    0
5170011    3
5170012    4
5170013    3
Name: items, Length: 5153209, dtype: int64

In [12]:
#Modifico tipo de dato, str --> int
def toInt(dic):
    dic['item_id']=int(dic['item_id'])

df_Users_Items['items'].apply(lambda x: toInt(x))

0          None
1          None
2          None
3          None
4          None
           ... 
5170009    None
5170010    None
5170011    None
5170012    None
5170013    None
Name: items, Length: 5153209, dtype: object

In [27]:
def itemsId_Null(fila):
    return fila['items']['item_name']==''

df_Users_Items.apply(itemsId_Null, axis=1).sum()

0

Consultas df_Users_Items

In [116]:
#Se encuentra algun item_name con el valor CS, Counter, Strike, Counter-Strike? 
def existe_CS(fila):
    return fila['item_name']=='Strike'

df_Users_Items['items'].apply(lambda x: existe_CS(x)).sum()

0

Lectura output_steam_games.json

In [80]:
df = pd.read_json('C:\\Users\\Usuario\\Desktop\\PI_MLOps\\dataset_original\\output_steam_games.json', lines=True)

Transform DataFrame

In [81]:
#Creo df unicamente con los siguientes campos: {genres, app_name, id}
df_Steam_Games = df[['genres','app_name','id']]

In [82]:
#Elimino nulos 
df_Steam_Games.dropna(inplace=True)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_7576\1082775196.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Steam_Games.dropna(inplace=True)


In [83]:
#Reset index
df_Steam_Games.reset_index(drop=True, inplace=True)

In [119]:
#Convertir a entero la comlumna 'id'
def to_Int(nro):
    try:
        #Try to convert
        return int(nro)
    except ValueError:
        #catch error
        return None
    
df_Steam_Games['id'] = df_Steam_Games['id'].apply(lambda x: to_Int(x)).dropna()

C:\Users\Usuario\AppData\Local\Temp\ipykernel_7576\1772122932.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Steam_Games['id'] = df_Steam_Games['id'].apply(lambda x: to_Int(x)).dropna()


In [120]:
df_Steam_Games

,genres,app_name,id
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,761140
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,643980
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,670290
3,"[Action, Adventure, Casual]",弹炸人2222,767400
4,"[Action, Adventure, Simulation]",Battle Royale Trainer,772540
...,...,...,...
28845,"[Action, Adventure, Casual, Indie]",Kebab it Up!,745400
28846,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,773640
28847,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,733530
28848,"[Indie, Racing, Simulation]",Russian Roads,610660


Consultas df_Steam_Games

In [136]:
#Tipo de dato, columna 'genres'
df_Steam_Games['genres'].describe()

count        28850
unique         883
top       [Action]
freq          1880
Name: genres, dtype: object

In [118]:
#Tipo de dato, columna 'id'
type(df_Steam_Games['id'][0])

numpy.float64

In [92]:
df_Steam_Games['app_name'].isna().sum()

0

In [107]:
#Busco juego 'Counter-Strike'
def existe_CS(fila):
    if (fila =='Strike'):
        return True 

df_Steam_Games['app_name'].apply(lambda x: existe_CS(x)).drop_duplicates()

0    None
Name: app_name, dtype: object

In [47]:
#Detalle del df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


In [137]:
#Detalles del df_Steam_Games
df_Steam_Games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28850 entries, 0 to 28849
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   genres    28850 non-null  object
 1   app_name  28850 non-null  object
 2   id        28850 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 676.3+ KB


In [50]:
#Cantidad de nulos
df_Steam_Games.isna().sum()

genres      91593
app_name    88312
id          88312
dtype: int64

In [57]:
#Verifico la eliminacion de los valores nulos

genres      0
app_name    0
id          0
dtype: int64